Best Practices
1. Preprocessing and cleaning
2. Train_Test_split
3. BPW,TFIDF,Word2vec
4. Choosing a ML algorithm

In [19]:
!pip install nltk
!pip install pandas
!pip install numpy
!pip install gensim
!pip install scikit-learn
!pip install bs4

In [108]:
import pandas as pd
df=pd.read_csv("all_kindle_review.csv")
df.head()


,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [109]:
df=df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [110]:
df.shape

(12000, 2)

In [111]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [112]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [113]:
df['rating']=df['rating'].apply(lambda x : 0 if x<3 else 1)

In [114]:
df['rating'].isnull().sum()

0

In [115]:
df['reviewText'].isnull().sum()

0

In [116]:
df['reviewText']=df['reviewText'].str.lower()


In [60]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GITESH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [117]:
#Remove special characters
df['reviewText']=df['reviewText'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+','',x))
#Remove stopwords
df['reviewText']=df['reviewText'].apply(lambda x : " ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
# ## Remove html tags
# df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


In [118]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [119]:
from nltk.stem import WordNetLemmatizer
lm=WordNetLemmatizer()

In [120]:
df['reviewText']=df['reviewText'].apply(lambda x: " ".join([lm.lemmatize(word) for word in x.split()]))

In [121]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


2. Train test split

In [67]:
from sklearn.model_selection import train_test_split
X_train_bow,X_test_bow,Y_train_bow,Y_test_bow=train_test_split(df['reviewText'],df['rating'],test_size=0.20,random_state=42)

In [68]:
X_train_bow.shape

(9600,)

In [78]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [79]:
X_train_bow_vector=cv.fit_transform(X_train_bow).toarray()
X_test_bow_vector=cv.transform(X_test_bow).toarray()

In [80]:
X_train_bow_vector.shape

(9600, 35726)

In [81]:
from sklearn.naive_bayes import GaussianNB
model_bow=GaussianNB()

In [82]:
model_bow.fit(X_train_bow_vector,Y_train_bow)

GaussianNB()

In [83]:
y_pred_bow=model_bow.predict(X_test_bow_vector)

In [84]:
from sklearn.metrics import accuracy_score,classification_report
print("Accuracy score: ",accuracy_score(Y_test_bow,y_pred_bow))
print("Classification report: ",classification_report(Y_test_bow,y_pred_bow))

Accuracy score:  0.5745833333333333
Classification report:                precision    recall  f1-score   support

           0       0.41      0.62      0.49       803
           1       0.74      0.55      0.63      1597

    accuracy                           0.57      2400
   macro avg       0.58      0.59      0.56      2400
weighted avg       0.63      0.57      0.59      2400



In [122]:
words=[]
for sent in df['reviewText']:
    words.append([word for word in sent.split()])
words[0]


['jace',
 'rankin',
 'may',
 'short',
 'he',
 'nothing',
 'mess',
 'man',
 'hauled',
 'saloon',
 'undertaker',
 'know',
 'he',
 'famous',
 'bounty',
 'hunter',
 'oregon',
 '1890s',
 'shot',
 'man',
 'saloon',
 'finished',
 'year',
 'long',
 'quest',
 'avenge',
 'sister',
 'murder',
 'trying',
 'figure',
 'next',
 'snotty-nosed',
 'farm',
 'boy',
 'rescued',
 'gang',
 'bully',
 'offer',
 'money',
 'kill',
 'man',
 'forced',
 'ranch',
 'reluctantly',
 'agrees',
 'bring',
 'man',
 'justice',
 'kill',
 'outright',
 'first',
 'need',
 'tell',
 'sister',
 'widower',
 'newskyla',
 'kyle',
 'springer',
 'bailey',
 'riding',
 'trail',
 'sleeping',
 'ground',
 'past',
 'month',
 'trying',
 'find',
 'jace',
 'want',
 'revenge',
 'man',
 'killed',
 'husband',
 'took',
 'ranch',
 'amongst',
 'crime',
 'shes',
 'keen',
 'detour',
 'jace',
 'want',
 'take',
 'realizes',
 'shes',
 'option',
 'hide',
 'behind',
 'boy',
 'persona',
 'best',
 'try',
 'keep',
 'pace',
 'confrontation',
 'along',
 'way',
 

In [128]:
from sklearn.model_selection import train_test_split
X_train_w2v,X_test_w2v,Y_train_w2v,Y_test_w2v=train_test_split(words,df['rating'],test_size=0.20,random_state=42)

In [129]:
len(X_train_w2v)

9600

In [130]:
from gensim.models import Word2Vec
model=Word2Vec(X_train_w2v,epochs=20,vector_size=200)

In [131]:
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'series',
 'much',
 'first',
 'well',
 'even',
 'didnt',
 'short',
 'could',
 'know',
 'way',
 'sex',
 'great',
 'make',
 'little',
 '-',
 'dont',
 'want',
 'thing',
 'two',
 'think',
 'find',
 'plot',
 'also',
 'romance',
 'end',
 'life',
 'im',
 'see',
 'enjoyed',
 'go',
 'scene',
 'never',
 'take',
 'written',
 'lot',
 'kindle',
 'many',
 'woman',
 'say',
 'thought',
 'work',
 'found',
 'bit',
 'novel',
 'going',
 'interesting',
 'liked',
 'year',
 'give',
 'loved',
 'feel',
 'writing',
 'better',
 'come',
 'another',
 'got',
 'man',
 'hot',
 'review',
 'enough',
 'still',
 'though',
 'back',
 'people',
 'made',
 'part',
 'something',
 'cant',
 'reader',
 'friend',
 'star',
 'page',
 'keep',
 'bad',
 'free',
 'need',
 'new',
 'wasnt',
 'doesnt',
 'recommend',
 'enjoy',
 'world',
 'relationship',
 'together',
 'next',
 'start',
 'felt',
 'best',
 

In [132]:
model.wv['king']

array([ 0.9336108 , -0.47133052,  0.45803392, -0.62823504, -0.6161148 ,
       -0.40042546,  0.7270098 ,  0.01770798,  0.03059852, -0.8384482 ,
        0.23717822,  0.61750335, -0.8964293 ,  0.5730252 , -0.5284956 ,
        0.3878737 ,  0.36932883,  0.07523521, -0.16604242,  0.02774365,
       -0.5668851 , -0.1984492 ,  0.41994244, -0.3179673 , -0.2913706 ,
       -0.5453013 , -0.40229726, -0.08604131,  0.04361576, -0.7277051 ,
        0.1573375 , -0.0085863 , -0.4466603 ,  0.36742976,  0.44443658,
        0.35117507,  0.36864743,  0.02414056,  0.22105603, -0.5885881 ,
        0.23114201,  0.10472544, -0.58036786,  0.5717759 ,  0.581798  ,
        0.41485828, -0.7743586 , -0.03625907, -0.22786243, -0.56114453,
       -0.3735545 ,  0.9546258 , -0.36716652,  0.2941159 ,  0.97075135,
       -1.2077541 ,  0.4424008 ,  0.44912347,  0.23113503, -0.0205315 ,
        0.11593719, -0.14814116, -0.07776902,  0.44508737, -0.7332887 ,
        0.38548574, -0.19259614, -0.47389126,  0.03414026, -0.15

In [133]:
import numpy as np
def avg_word2vec(doc):
    return np.mean([ model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [134]:
avg_word2vec_vec=[]
for word in X_train_w2v:
    avg_word2vec_vec.append(avg_word2vec(word))
    

In [152]:
avg_word2vec_vec_test=[]
for word in X_test_w2v:
    avg_word2vec_vec_test.append(avg_word2vec(word))

In [135]:
len(avg_word2vec_vec)

9600

In [136]:
len(avg_word2vec_vec[0])

200

In [154]:
df_w2v=pd.DataFrame([ x.reshape(1,-1)[0]for x in avg_word2vec_vec
])

df_test_w2v=pd.DataFrame([ x.reshape(1,-1)[0]for x in avg_word2vec_vec_test
])

In [155]:
df_test_w2v.shape

(2400, 200)

In [150]:
df_w2v.shape

(9600, 200)

In [ ]:
df_w2v.isnull().sum()

0      0
1      0
2      0
3      0
4      0
      ..
195    0
196    0
197    0
198    0
199    0
Length: 200, dtype: int64

In [145]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()

In [148]:
rfc.fit(df_w2v,Y_train_w2v)

RandomForestClassifier()

In [156]:
y_pred_w2v=rfc.predict(df_test_w2v)

In [158]:
from sklearn.metrics import accuracy_score,classification_report
print("Accuracy score: ",accuracy_score(Y_test_w2v,y_pred_w2v))
print("Classification report: ",classification_report(Y_test_w2v,y_pred_w2v))

Accuracy score:  0.80125
Classification report:                precision    recall  f1-score   support

           0       0.77      0.58      0.66       803
           1       0.81      0.91      0.86      1597

    accuracy                           0.80      2400
   macro avg       0.79      0.75      0.76      2400
weighted avg       0.80      0.80      0.79      2400

